<a href="https://colab.research.google.com/github/samuelshoun/Useful-MWAs/blob/main/differential_evolution_with_integrality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Differential evolution using scipy-optimize is my go-to method for curve fitting, as shown in this minimal working example.

The docs for DE are <a href=https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html> here</a>.

Actually, this example goes somewhat beyond minimal by demonstrating the use of the `integrality` argument to allowing for estimators (functions of x) within the objective function. This has proved very useful in engineering tasks where it's uncertain whether a particular line or segment might better be fit with a linear versus a log function, for instance. 

In [1]:
import numpy as np
from scipy.optimize import differential_evolution

In [2]:
# define a data-generating function
# in this case, a simple linear function of x

def func(x, m, b):
    y = m * x + b
    return y

In [3]:
# define the x space
x = np.linspace(0, 1, 100)

# choose true parameter values & calculate the function values
m = 2
b = 5
y = func(x, m, b)

In [4]:
# define an objective function
# CRITICAL NOTE:
# The parameters being optimized are sent to the loss function
# as a SINGLE ITERABLE ARGUMENT (`args` in this case).

def obj(args, x_data):
    # use if-else and the parameter set for 
    # integrality in the differential_evolution args
    if args[2] == 1:
        yhat = func(x_data, args[0], args[1])     # use the data generating function
    else:
        yhat = x_data + 100                       # use some other function

    mae = np.sum(abs(yhat - y)) / len(y)
    
    return mae

In [5]:
# example objective calcs
obj([2, 5, 1], x), obj([2, 5, 0], x)

(0.0, 94.5)

In [6]:
# define boundaries for the parameter space
bounds = [(-100, 100), (-100, 100), (0, 1)]

In [7]:
# execute the optimization using differential evolution
# CRITICAL NOTE:
# for a single argument passed through 'differential_evolution' argument
# 'args': `args=(x)` will not work, but `args=(x, )` will work.

%%time
de = differential_evolution(obj, bounds=bounds, args=(x, ), integrality=[0,0,1])
de

CPU times: user 1.02 s, sys: 6.06 ms, total: 1.03 s
Wall time: 1.45 s


 message: Optimization terminated successfully.
 success: True
     fun: 5.329070518200751e-15
       x: [ 2.000e+00  5.000e+00  1.000e+00]
     nit: 151
    nfev: 6903

The algorithm quickly converges on a the correct parameters (see attribute `x` in the output). The final parameter in `x` is the one selected for integrality, and controls which estimator is used within the objective function.

In [8]:
# predict using the optimized parameters
func(x, de.x[0], de.x[1])

array([5.        , 5.02020202, 5.04040404, 5.06060606, 5.08080808,
       5.1010101 , 5.12121212, 5.14141414, 5.16161616, 5.18181818,
       5.2020202 , 5.22222222, 5.24242424, 5.26262626, 5.28282828,
       5.3030303 , 5.32323232, 5.34343434, 5.36363636, 5.38383838,
       5.4040404 , 5.42424242, 5.44444444, 5.46464646, 5.48484848,
       5.50505051, 5.52525253, 5.54545455, 5.56565657, 5.58585859,
       5.60606061, 5.62626263, 5.64646465, 5.66666667, 5.68686869,
       5.70707071, 5.72727273, 5.74747475, 5.76767677, 5.78787879,
       5.80808081, 5.82828283, 5.84848485, 5.86868687, 5.88888889,
       5.90909091, 5.92929293, 5.94949495, 5.96969697, 5.98989899,
       6.01010101, 6.03030303, 6.05050505, 6.07070707, 6.09090909,
       6.11111111, 6.13131313, 6.15151515, 6.17171717, 6.19191919,
       6.21212121, 6.23232323, 6.25252525, 6.27272727, 6.29292929,
       6.31313131, 6.33333333, 6.35353535, 6.37373737, 6.39393939,
       6.41414141, 6.43434343, 6.45454545, 6.47474747, 6.49494